In [1]:
# The program aims to generate the html file from the question skeleton
# It is different from normal question relations since it's procedural

import networkx as nx 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json 
import pickle
import os
from shutil import copyfile

from IPython.core.debugger import set_trace

In [ ]:
# simply define some skeletons here
"""skeleton = ['Why is it...?', 'What is the hypothesis?','What are the empirical observation?','Is the hypothesis good at explaining the empirical observation?']
pagetitle = 'why question skeleton'"""


"""skeleton = ['How to solve this?','What is the proposed solution?','Is the solution good?','How to improve this solution?']
pagetitle = 'how question skeleton'"""

"""skeleton = ['Is this good?','What are effects of this?','Who benefits from / is harmed by these effects?','How to eliminate the negative effects?']
pagetitle = 'evaluation question skeleton'"""


# specify vis properties
nd_color = '#FFC38B'
nd_size = 5
nd_shape='dot'
lk_color = '#FFF3CD'
lk_width = 3 #much thicker than the others
# construct the network
ht_nodes,ht_edges = [],[]
for kn,node in enumerate(skeleton):
    ht_nodes.append({'id':kn, 'label': wrap_by_word(node,5),'fulllabel':node, 'type':'skeletonQ',
                     'color': nd_color, 'size':nd_size,'shape':nd_shape, 'font':{'vadjust':'25'},
                    'x':kn*160,'y':kn%2*70}) # specify locations of this chain (wavy y to save space)
    if kn>0:
        ht_edges.append({'from':kn-1, 'to': kn, 'label': '', 'color':lk_color,
                     'font':{'align':'middle'},'arrows':'to','width':lk_width,'style':"dashed",'arrowtail':'vee'})

In [79]:

skeleton = ['Why is it...?', 'What are the hypotheses?','What are the empirical observation?','Is the hypothesis good at explaining the empirical observation?','Which of the hypotheses explain the empirical observation best?']
expansion = [[],['hypothesis 1','hypothesis 2'],['experiment a','experiment b'],['hyp 1-exp a','hyp 1-exp b','hyp 2-exp a','hyp 2-exp b'],[]]#
pagetitle = 'why question expanded skeleton'
# specify vis properties
nd_color = '#FFC38B'
nd_size = 5
nd_shape='dot'
expnd_color = '#C3EDEA'
explk_width = 1.5
lk_color = '#FFF3CD'
lk_width = 3 #much thicker than the others
# construct the network
ht_nodes,ht_edges = [],[]
for kn,node in enumerate(skeleton):
    ht_nodes.append({'id':str(kn), 'label': wrap_by_word(node,5),'fulllabel':node, 'type':'skeletonQ',
                     'color': nd_color, 'size':nd_size,'shape':nd_shape, 'font':{'vadjust':'25'},
                    'x':kn*160,'y':kn%2*70}) # specify locations of this chain (wavy y to save space)
    if kn>0:
        ht_edges.append({'from':kn-1, 'to': kn, 'label': '', 'color':lk_color,
                     'font':{'align':'middle'},'arrows':'to','width':lk_width,'style':"dashed",'arrowtail':'vee'})
    if len(expansion[kn]):
        for kex,expnd in enumerate(expansion[kn]):
            ht_nodes.append({'id':str(kn)+'_'+str(kex), 'label': wrap_by_word(expnd,5),'fulllabel':expnd, 'type':'skeleton_expansion',
                     'color': expnd_color, 'size':nd_size,'shape':nd_shape, 'font':{'vadjust':'25'},
                    'x':kn*160+10+20*kex,'y':kn%2*70-100+40*kex
                            }) # specify locations of this chain (wavy y to save space)
            ht_edges.append({'from':kn, 'to': str(kn)+'_'+str(kex), 'label': '', 'color':expnd_color,
                     'font':{'align':'middle'},'arrows':'to','width':explk_width,'style':"dashed",'arrowtail':'vee'})
            
            
            

In [81]:
# output the html by revising the template    
htmltitle = pagetitle
pageinfo =''
tfile = 'html_templates/html_skeleton.html' # template file
rawname = pagetitle+'.html'
with open(tfile, 'r') as f,open(rawname, 'w') as rawf:
        content = f.readlines()
        for kl,line in enumerate(content):            
            if kl==5:
                rawf.write('<TITLE>'+pagetitle+'</TITLE>')
            elif kl==74:
                rawf.write('var nodes = new vis.DataSet('+str(ht_nodes)+')\n')
            elif kl==75:
                rawf.write('var edges = new vis.DataSet('+str(ht_edges)+')\n')
            else:
                rawf.write(line)

In [2]:
def wrap_by_word(s, n):
    # returns a string where \\n is inserted between every n words
    a = s.split()
    wrapped = ''
    for i in range(0, len(a), n):
        wrapped += ' '.join(a[i:i+n]) + '\n'
    return wrapped